In [38]:
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt

from time import time                                # Record computation time
from numpy.fft import fft, ifft, fftshift, ifftshift # Fast Fourier Transform
from scipy.interpolate import interp1d               # Interpolation

plt.rcParams['axes.axisbelow'] = True   # Set axes and grid elements to be below the figure

In [39]:
# Monte Carlo Option Pricing ==========================================================================================
# Contract parameters
T = 1  # maturity
K = 1.1  # strike price

# Market parameters
S0 = 1  # initial stock price
r = 0.5  # risk-free interest rate
q = 0.3  # dividend rate

MU = r - q   # drift

# Model parameters
KAPPA = 3    # mean-reversion rate
THETA = 0.3  # mean-reversion level
SIGMA = 0.25 # volatility of volatility
V0 = 0.08    # initial volatility
RHO = -0.8   # correlation of W1 and W2

# Simulation parameters
N_STEPS = 200                      # number of time steps
N_BLOCKS = 100                     # number of blocks
N_SIMS = 20000                     # number of paths per block

dt = T / N_STEPS                   # time step
t = np.linspace(0, T, N_STEPS + 1) # time grid

In [40]:
# def fft_Lewis(K, S0, cf, interp="cubic"):
#     """
#     K = vector of strike
#     S = spot price scalar
#     cf = characteristic function
#     interp can be cubic or linear
#     """
#     N = 2**12  # FFT more efficient for N power of 2
#     B = 200  # integration limit
#     dx = B / N
#     x = np.arange(N) * dx  # the final value B is excluded

#     weight = np.arange(N)  # Simpson weights
#     weight = 3 + (-1) ** (weight + 1)
#     weight[0] = 1
#     weight[N - 1] = 1

#     dk = 2 * np.pi / B
#     b = N * dk / 2
#     ks = -b + dk * np.arange(N)

#     integrand = np.exp(-1j * b * np.arange(N) * dx) * cf(x - 0.5j) * 1 / (x**2 + 0.25) * weight * dx / 3
#     integral_value = np.real(ifft(integrand) * N)

#     if interp == "linear":
#         spline_lin = interp1d(ks, integral_value, kind="linear")
#         prices = S0 - np.sqrt(S0 * K) * np.exp(-r * T) / np.pi * spline_lin(np.log(S0 / K))
#     elif interp == "cubic":
#         spline_cub = interp1d(ks, integral_value, kind="cubic")
#         prices = S0 - np.sqrt(S0 * K) * np.exp(-r * T) / np.pi * spline_cub(np.log(S0 / K))
#     return prices
    

In [41]:
# # # FOURIER SOLUTION ===============================================================================================
# temp = time()

# # Controls


# def payoff(x, xi, ALPHA, K, L, U, C, theta):
#     # Scale
#     S = C * np.exp(x)

#     # Payoff; see e.g. Green, Fusai, Abrahams 2010, Eq. (3.24)
#     g = np.exp(ALPHA * x) * np.maximum(theta * (S - K), 0) * (S >= L) * (S <= U)

#     # Analytical Fourier transform of the payoff
#     l = np.log(L / C)  # lower log barrier
#     k = np.log(K / C)  # log strike
#     u = np.log(U / C)  # upper log barrier

#     # Integration bounds
#     if theta == 1:  # call
#         a = max(l, k)
#         b = u
#     else:  # put
#         a = min(k, u)
#         b = l

#     # Green, Fusai, Abrahams 2010 Eq. (3.26) with extension to put option
#     xi2 = ALPHA + 1j * xi
#     G = C * ((np.exp(b * (1 + xi2)) - np.exp(a * (1 + xi2))) / (1 + xi2) - (np.exp(k + b * xi2) - np.exp(k + a * xi2)) / xi2)

#     # Eliminable discontinuities for xi = 0, otherwise 0/0 = NaN
#     if ALPHA == 0:
#         G[len(G) // 2] = C * (np.exp(b) - np.exp(a) - np.exp(k) * (b - a))
#     elif ALPHA == -1:
#         G[len(G) // 2] = C * (b - a + np.exp(k - b) - np.exp(k - a))

#     return S, g, G

# # Grids in real and Fourier space
# N = N_GRIDS // 2
# b = X_WIDTH / 2  # upper bound of the support in real space
# dx = X_WIDTH / N_GRIDS  # grid step in real space
# x = dx * np.arange(-N, N)  # grid in real space
# dxi = np.pi / b  # Nyquist relation: grid step in Fourier space
# xi = dxi * np.arange(-N, N)  # grid in Fourier space

# # Characteristic function at time T
# muABM = R - Q - 0.5 * SIGMA**2 # drift coefficient of the arithmetic Brownian motion

# xia = xi + 1j * ALPHA  # call
# psi = 1j * muABM * xia - 0.5 * (SIGMA * xia) ** 2  # characteristic exponent
# psi_call = np.exp(psi * T)  # characteristic function
# xia = xi - 1j * ALPHA  # put
# psi = 1j * muABM * xia - 0.5 * (SIGMA * xia) ** 2  # characteristic exponent
# psi_put = np.exp(psi * T)  # characteristic function

# # Fourier transform of the payoff
# b = X_WIDTH / 2  # upper bound of the support in real space
# U = S0 * np.exp(b)
# L = S0 * np.exp(-b)
# _, gc, Gc = payoff(x, xi, ALPHA, K, L, U, S0, 1)  # call
# S, gp, Gp = payoff(x, xi, -ALPHA, K, L, U, S0, 0)  # put

# # Discounted expected payoff computed with the Plancherel theorem
# c = np.exp(-R * T) * np.real(fftshift(fft(ifftshift(Gc * np.conj(psi_call))))) / X_WIDTH  # call
# call_fourier = interp1d(S, c, kind='cubic')(S0)
# p = np.exp(-R * T) * np.real(fftshift(fft(ifftshift(Gp * np.conj(psi_put))))) / X_WIDTH  # put
# put_fourier = interp1d(S, p, kind='cubic')(S0)

# # # ================================================================================================================
# t_fourier = time() - temp

In [42]:
# SIMULATION ==========================================================================================================
temp = time()

call_mc_blocks = np.zeros(N_BLOCKS)
put_mc_blocks = np.zeros(N_BLOCKS)

for j in range(N_BLOCKS): # We change the index to ensure the inner loop is not inturrupted
    
    Z1 = np.random.randn(N_STEPS, N_SIMS)
    Z2 = RHO * Z1 + np.sqrt(1 - RHO**2) * np.random.randn(N_STEPS, N_SIMS)

    # Initialize the price and volatility processes
    V = np.vstack([np.full(N_SIMS, V0), np.zeros((N_STEPS, N_SIMS))])
    
    # Volatility process
    a = SIGMA**2 / KAPPA * (np.exp(-KAPPA * dt) - np.exp(-2 * KAPPA * dt))
    b = THETA * SIGMA**2 / (2 * KAPPA) * (1 - np.exp(-KAPPA * dt))**2
    for i in range(N_STEPS):
        # THETA + (V[i, :] - THETA) * np.exp(-KAPPA * dt) + np.sqrt(a * V[i] + b) * Z2[i]
        V[i + 1] = THETA * (1 - np.exp(-KAPPA * dt)) + V[i] * np.exp(-KAPPA * dt) + np.sqrt(a * V[i] + b) * Z2[i]
        V[i + 1] = np.maximum(V[i + 1], 0)

    # Price process
    S = (MU - 0.5 * V[:-1]) * dt + np.sqrt(V[:-1] * dt) * Z1
    S = np.vstack([np.zeros(N_SIMS), S.cumsum(axis = 0)])
    S = S0 * np.exp(S[-1]) # We are only interested in the terminal value for European options

    # Compute the discounted payoff
    call_mc_blocks[j] = np.exp(-r * T) * np.mean(np.maximum(S - K, 0))
    put_mc_blocks[j] = np.exp(-r * T) * np.mean(np.maximum(K - S, 0))



call_mc = call_mc_blocks.mean()
put_mc = put_mc_blocks.mean()

call_mc_se = call_mc_blocks.std() / np.sqrt(N_BLOCKS)
put_mc_se = put_mc_blocks.std() / np.sqrt(N_BLOCKS)

t_mc = time() - temp

In [43]:
# PRINT RESULTS ===================================================================================================
print(f"{'':18s}{'Call':15s}{'Put':15s}{'CPU Time/s':15s}{'Operations':15s}")
# print(f"{'Fourier':15s}{call_fourier:15.10f}{put_fourier:15.10f}{t_fourier:15.10f}{N_GRIDS:13d}")
print(f"{'Monte Carlo':15s}{call_mc:15.10f}{put_mc:15.10f}{t_mc:15.10f}{N_BLOCKS * N_SIMS:13d}")
print(f"{'MC S.E.':15s}{call_mc_se:15.10f}{put_mc_se:15.10f}")

                  Call           Put            CPU Time/s     Operations     
Monte Carlo       0.1728389727   0.0992537446  28.7916121483      2000000
MC S.E.           0.0002229142   0.0000852768


In [44]:
call_mc - put_mc - S0 * np.exp(-q * T) + K * np.exp(-r * T)

-4.926692292206436e-05